# Урок 2

В этом уроке мы реализуем DTW алгоритм на основе token passing algorithm (TPA).

Начнем с того, что у нас есть несколько эталонов (высказываний) и одна запись. Необходимо определить, к какому из эталонов наша запись "ближе" с точки зрения дистанции (сумарного евклидовго расстояния), полученной с помощью DTW.

Для тренировки будем использовать искусственные одномерные признаки:

запись [<br>
0<br>
0.9<br>
1.3<br>
0<br>
]<br>

эталон-да [<br>
0<br>
1<br>
2<br>
0<br>
]

эталон-нет [<br>
0<br>
1<br>
0<br>
]

Они уже записаны в формате ark и хранятся в файлах record_mfcc.txtftr и etalons_mfcc.txtftr соответственно. 

Для начала нужно реализовать функцию load_graph, которая построит граф на основе используемых эталонов. Каждая его ветвь будет одним из эталонов. Эталоны состоят из узлов типа State. Класс State имеет следующие атрибуты:

- ftr – вектор признаков узла;<br>
- isFinal – является ли этот узел финальным в слове;<br>
- word – слово эталона (назначается только для финального узла ветви);<br> 
- nextState – список узлов, к которым в данном узле есть переход;<br>
- idx – индекс узла;<br>

На данном этапе узлы в графе будут иметь переходы только в себя и следующий state. Нулевой узел является корневым и имеет переходы в начальные узлы каждой ветви. Финальный узел каждой ветви имеет переход только в себя.

Такой граф представлен ниже:

<img src="graph_1.jpg">

<br>
Теперь попробуем реализовать такой граф:

In [2]:
import time
import numpy as np
import FtrFile

class State:
    def __init__(self, ftr, idx):  
        self.ftr = ftr           
        self.isFinal = False     
        self.word = None               
        self.nextStates = []     
        self.idx = idx           

def load_graph(rxfilename):
    startState = State(None, 0)
    graph = [startState, ]
    stateIdx = 1
    for word, features in FtrFile.FtrDirectoryReader(rxfilename):
        prevState = startState
        for frame in range(features.nSamples):
            state = State(features.readvec(), stateIdx)
            state.nextStates.append(state)  # add loop
            prevState.nextStates.append(state)
            prevState = state 
            graph.append(state)
            stateIdx += 1
        if state:
            state.word = word
            state.isFinal = True
    return graph

Далее идет описание класса Token и самого алгоритма TPA:

- Алгоритм TPA двигается последовательно по кадрам записи и на каждом кадре берёт множество токенов от предыдущего кадра, и порождает множество токенов для текущего кадра.
- Один токен – это штука, олицетворяющая собой один из возможных вариантов разметки (соотнесения кадров записи кадрам эталона), заканчивающаяся на данном кадре. По токену должно быть можно понять, какую суммарную дистанцию набрал данный токен (то есть то, насколько он хорош), в какой узле графа эталонов он находится (чтобы от него можно было породить токен для следующего кадра), и через какие слова он прошёл (чтобы по лучшему токену можно было получить результат).
- На некотором кадре всё множество токенов описывает все возможные разметки, которые можно получить к данному кадру. После обработки последнего кадра, мы просто переберём все финальные токены (разметки) и выберем лучший (та, которая имеет минимальное суммарное расстояние от записи до эталона).
- Финальные токены – это токены, которым соответствует законченная разметка (то есть та, в которой после финального кадра мы оказались на финальном узле графа). Остальные токены – фактически, бракованные.

Получается, что TPA (в том виде,в котором он тут описан) – это удобная форма записи полного перебора всех возможных разметок.

Теперь определим класс Token, функцию distance для вычисления евклидова расстояния между двумя векторами и главную функцию recognize, в которой и определяется наилучший токен для текущей записи, посредством работы TPA.

In [3]:
class Token:
    def __init__(self, state, dist=0.0, sentence=""):
        self.state = state        # узел графа, в котором находится токен
        self.dist = dist          # общая дистанция токена
        self.sentence = sentence  # накопленное высказывание (в данном случае это просто слово эталона)

# функция вычисления евклидова расстояния между двумя векторами:        
def distance(X, Y):
    #------------------------TODO-----------------------------
    result = float(np.sqrt(pow(X - Y, 2))) 
    #---------------------------------------------------------
    return result


def recognize(filename, features, graph):
    print("Recognizing file '{}', samples={}".format(filename, features.nSamples))

    startState = graph[0]
    activeTokens = [Token(startState), ]
    nextTokens = []

    for frame in range(features.nSamples):
        ftrCurrentFrameRecord = features.readvec()
        for token in activeTokens:
            for transitionState in token.state.nextStates:
                #-------------------- TODO ------------------------------------------
                # 1. создаем новый токен
                # 2. вычисляем дистанцию токена для текущего кадра записи
                # 3. добавляем к nextTokens созданный новый токен
                newToken = Token(transitionState, token.dist, token.sentence)
                newToken.dist += distance(ftrCurrentFrameRecord, transitionState.ftr)
                nextTokens.append(newToken)
                #--------------------------------------------------------------------
        activeTokens = nextTokens
        nextTokens = []

    # поиск финальных токенов:
    finalTokens = []
    for token in activeTokens:
        #----------------- TODO -------------------------
        if token.state.isFinal:
            finalTokens.append(token)
        #------------------------------------------------
    
    # поиск наилучшего финального токена:
    winToken = None
    #-------------------- TODO -----------------------
    minDist = finalTokens[0].dist
    for token in finalTokens:
        if token.dist <= minDist:
            winToken = token
            minDist = token.dist
    #-------------------------------------------------
    
    # отображение параметров победившего токена:
    print('-'*50)
    print("WIN TOKEN: state.word = '{}', dist = {}, ".format(winToken.state.word, round(winToken.dist, 3)))
    print('-' * 50)

Описание всех классов и функций окончено, остается только запустить нашу программу.

In [4]:
etalons = "ark,t:etalons_mfcc.txtftr"   # файл с признаками эталонов
record = "ark,t:record_mfcc.txtftr"    # файл с признаками записи

# загрузка графа:
graph = load_graph(etalons)

for filename, features in FtrFile.FtrDirectoryReader(record):
    recognize(filename, features, graph)

Recognizing file 'record1', samples=4
--------------------------------------------------
WIN TOKEN: state.word = 'etalon_Net', dist = 0.4, 
--------------------------------------------------


Для проверки посчитайте DTW в ручную и сравните полученный результат.